In [1]:
import re
import email
from time import asctime
import os
import sys
from dateutil.parser import parse

In [7]:
maildir = 'maildir/'
MBOX = 'enron.mbox'

mbox = open(MBOX, 'w+')

In [8]:
for (root, dirs, file_names) in os.walk(maildir):
    if root.split(os.sep)[-1].lower() != 'inbox':
        continue
    for file_name in file_names:
        file_path = os.path.join(root, file_name)
        message_text = open(file_path, errors='ignore').read()
        
        # Определить поля для строки From_ в типичном сообщении mbox
        _from = re.search(r"From: ([^\r\n]+)", message_text).groups()[0]
        _date = re.search(r"Date: ([^\r\n]+)", message_text).groups()[0]
        
        # Преобразовать _date в представление asctime для строки From_
        _date = asctime(parse(_date).timetuple())
        
        msg = email.message_from_string(message_text)
        msg.set_unixfrom(f'From {_from} {_date}')
        
        mbox.write(msg.as_string(unixfrom=True) + "\n\n")
mbox.close()

In [9]:
import pandas as pd
import mailbox

In [10]:
MBOX = 'enron.mbox'
mbox = mailbox.mbox(MBOX)

In [12]:
mbox_dict = {}
for i, msg in enumerate(mbox):
    mbox_dict[i] = {}
    for header in msg.keys():
        mbox_dict[i][header] = msg[header]
    mbox_dict[i]['Body'] = msg.get_payload().replace('\n',' ').replace('\t', ' ').replace('\r', ' ').strip()
    
df = pd.DataFrame.from_dict(mbox_dict, orient='index')
df.head()

,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,...,X-FileName,Body,Cc,Bcc,http,Re,dal,"""Notice",X-Mailer,<http
0,<16159836.1075855377439.JavaMail.evans@thyme>,"Fri, 7 Dec 2001 10:06:42 -0800 (PST)",heather.dunton@enron.com,k..allen@enron.com,RE: West Position,1.0,text/plain; charset=us-ascii,7bit,"Dunton, Heather </O=ENRON/OU=NA/CN=RECIPIENTS/...","Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...",...,pallen (Non-Privileged).pst,Please let me know if you still need Curve Shi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,<14955894.1075855377681.JavaMail.evans@thyme>,"Sun, 30 Dec 2001 22:49:42 -0800 (PST)",anchordesk_daily@anchordesk.zdlists.com,pallen@enron.com,ANCHORDESK: Hope ahead: What I learned from 20...,1.0,text/plain; charset=ANSI_X3.4-1968,7bit,"""AnchorDesk"" <AnchorDesk_Daily@anchordesk.zdli...",pallen@ENRON.COM,...,pallen (Non-Privileged).pst,_____________________DAVID COURSEY____________...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,<7462038.1075855377703.JavaMail.evans@thyme>,"Sun, 30 Dec 2001 23:42:30 -0800 (PST)",subscriptions@intelligencepress.com,pallen@enron.com,"NGI Publications - Monday, December 31st 2001",1.0,text/plain; charset=us-ascii,7bit,subscriptions@intelligencepress.com@ENRON,pallen@enron.com,...,pallen (Non-Privileged).pst,"Dear phillip, This e-mail is automated notif...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,<21572157.1075855377726.JavaMail.evans@thyme>,"Mon, 31 Dec 2001 02:24:51 -0800 (PST)",prizemachine@feedback.iwon.com,pallen@enron.com,"Click. Spin. Chances to Win up to $10,000!",1.0,text/plain; charset=us-ascii,7bit,Prize Machine<PrizeMachine@feedback.iwon.com>@...,pallen@enron.com,...,pallen (Non-Privileged).pst,"[IMAGE] [IMAGE] [IMAGE] [IMAGE] $ 2,500 [I...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,<7618763.1075855377753.JavaMail.evans@thyme>,"Mon, 31 Dec 2001 10:53:43 -0800 (PST)",louise.kitchen@enron.com,"wes.colwell@enron.com, georgeanne.hodges@enron...",NETCO,1.0,text/plain; charset=us-ascii,7bit,"Kitchen, Louise </O=ENRON/OU=NA/CN=RECIPIENTS/...","Colwell, Wes </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",...,pallen (Non-Privileged).pst,The New Year has arrived and we really to fina...,john.lavorato@enron.com,john.lavorato@enron.com,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df.index = df['Date'].apply(pd.to_datetime)

df = df[['From', 'To', 'Cc', 'Bcc', 'Subject', 'Body']]
df.head()

,From,To,Cc,Bcc,Subject,Body
Date,,,,,,
2001-12-07 10:06:42-08:00,heather.dunton@enron.com,k..allen@enron.com,NaN,NaN,RE: West Position,Please let me know if you still need Curve Shi...
2001-12-30 22:49:42-08:00,anchordesk_daily@anchordesk.zdlists.com,pallen@enron.com,NaN,NaN,ANCHORDESK: Hope ahead: What I learned from 20...,_____________________DAVID COURSEY____________...
2001-12-30 23:42:30-08:00,subscriptions@intelligencepress.com,pallen@enron.com,NaN,NaN,"NGI Publications - Monday, December 31st 2001","Dear phillip, This e-mail is automated notif..."
2001-12-31 02:24:51-08:00,prizemachine@feedback.iwon.com,pallen@enron.com,NaN,NaN,"Click. Spin. Chances to Win up to $10,000!","[IMAGE] [IMAGE] [IMAGE] [IMAGE] $ 2,500 [I..."
2001-12-31 10:53:43-08:00,louise.kitchen@enron.com,"wes.colwell@enron.com, georgeanne.hodges@enron...",john.lavorato@enron.com,john.lavorato@enron.com,NETCO,The New Year has arrived and we really to fina...


In [17]:
# Получение списка рейтингов отправителей и получателей в корпусе Enron
import numpy as np

top_senders = df.groupby('From')
top_receivers = df.groupby('To')

top_senders = top_senders.count()['To']
top_receivers = top_receivers.count()['From']

# Получить индексы отправителей и получателей в порядке убывания
top_snd_ord = np.argsort(top_senders)[::-1]
top_rcv_ord = np.argsort(top_receivers)[::-1]

top_senders = top_senders[top_snd_ord]
top_receivers = top_receivers[top_rcv_ord]

In [23]:
# Поиск 10 самых активных отправителей в корпусе Enron
from prettytable import PrettyTable

top10 = top_senders[:10]
pt = PrettyTable(field_names=['Rand', 'Sender', 'Message Sent'])
pt.align['Message Sent'] = 'r'
[pt.add_row([i+1, email, vol]) for i, email, vol in zip(range(10), top10.index.values, top10.values)]
print(pt)

+------+-----------------------------------+--------------+
| Rand |               Sender              | Message Sent |
+------+-----------------------------------+--------------+
|  1   |        pete.davis@enron.com       |          722 |
|  2   |   announcements.enron@enron.com   |          372 |
|  3   |        jae.black@enron.com        |          322 |
|  4   | enron_update@concureworkplace.com |          213 |
|  5   |         feedback@intcx.com        |          209 |
|  6   |       chairman.ken@enron.com      |          197 |
|  7   |     arsystem@mailman.enron.com    |          192 |
|  8   |       mike.grigsby@enron.com      |          191 |
|  9   |       soblander@carrfut.com       |          186 |
|  10  |        mary.cook@enron.com        |          186 |
+------+-----------------------------------+--------------+


In [24]:
# Поиск 10 самых активных получателей в корпусе Enron
top10 = top_receivers[:10]
pt = PrettyTable(field_names=['Rank', 'Receiver', 'Messages Received'])
pt.align['Message Sent'] = 'r'
[pt.add_row([i+1, email, vol]) for i, email, vol in zip(range(10), top10.index.values, top10.values)]
print(pt)

+------+------------------------------+-------------------+
| Rank |           Receiver           | Messages Received |
+------+------------------------------+-------------------+
|  1   |     pete.davis@enron.com     |        721        |
|  2   |    gerald.nemec@enron.com    |        677        |
|  3   |    kenneth.lay@enron.com     |        608        |
|  4   |  sara.shackleton@enron.com   |        453        |
|  5   |   jeff.skilling@enron.com    |        420        |
|  6   | center.dl-portland@enron.com |        394        |
|  7   |   jeff.dasovich@enron.com    |        346        |
|  8   |     tana.jones@enron.com     |        303        |
|  9   |      rick.buy@enron.com      |        286        |
|  10  |   barry.tycholiz@enron.com   |        280        |
+------+------------------------------+-------------------+


In [27]:
import textwrap

search_term = 'raptor'

query = (df['Body'].str.contains(search_term, case=False) | df['Subject'].str.contains(search_term, case=False))

results = df[query]

print('{0} results found.'.format(query.sum()))
print('Printing first 10 results...')
for i in range(10):
    subject, body = results.iloc[i]['Subject'], results.iloc[i]['Body']
    print()
    print('SUBJECT: ', subject)
    print('-'*20)
    for line in textwrap.wrap(body, width=70, max_lines=5):
        print(line)

78 results found.
Printing first 10 results...

SUBJECT:  RE: Raptor Debris
--------------------
I think that all investments that were hedged in Raptor are on th MPR
and are being monitored and reported against.  Is this true?  Ron,
will you take the lead in making sure we have a smooth transition on
this?  Thanks  -----Original Message----- From:  Buy, Rick   Sent:
Wednesday, October 03, 2001 9:38 AM To: Causey, Richard; Glisan, [...]

SUBJECT:  FW: Raptor Debris
--------------------
Rick, Let me know if you would like to get together to discuss how the
Raptor unwind is accomplished from a mechanical standpoint, as well
as, what comes back to Enron in the transaction.  I would happy to sit
down and walk you through it.  Just let me know when might work for
you to get together.  Thanks Rick, Ron Baker   -----Original [...]

SUBJECT:  LJM/Raptor valuations
--------------------
Ryan,  In the follow up to the meeting we had on Wednesday I would
like to reinforce one point I made. I feel 